In [1]:
from grandma_stock_valuation import FileLogger, YahooDataLoader
from grandma_stock_valuation import batchValuation, addCashPortfolio, getCorrelationWeight, allocatePortfolio
import time
import pandas as pd

logger = FileLogger()
logPrint = logger.logPandas


### Refresh Data

    'IXN':'Global Tech',
    'XT':'Global Tech',
    'QQQ':'Nasdaq 100',
    'IXG':'Global Finance',
    'IXJ':'Global Healthcare',
    'KXI':'Global Consumer Staple',
    'RXI':'Global Consumer Discretionary',

In [2]:
LOAD_SAVED_DATA = True

TIME_SLEEP = 0 if LOAD_SAVED_DATA else 0.5

d_instrument = {
    'IVV':'SP500',
    '3073.HK':'Greater China',
    'VPL':'Developed Asia-Pacific',
    'IEV':'Europe',
    'ASEA':'SE Asia',
    
    #'MOAT':'US Moat',
    #'AAXJ':'Asia ex Japan',
    #'EZU':'Eurozone'
}

d_instrument_data = {}
for ticker, name in d_instrument.items():
    logPrint(f"{ticker}: {name}")

    if LOAD_SAVED_DATA:
        df = pd.read_csv(f'_data/{ticker}_EOD.csv.gz')
        df['date'] = pd.to_datetime(df['date'])
    else:
        yahoo = YahooDataLoader(ticker, verbose=0, printfunc=logPrint)
        df = yahoo.queryEOD(save=True)

    d_instrument_data[ticker] = df
    time.sleep(TIME_SLEEP)


2022-03-15 07:58:03,136 INFO IVV: SP500
2022-03-15 07:58:03,154 INFO 3073.HK: Greater China
2022-03-15 07:58:03,162 INFO VPL: Developed Asia-Pacific
2022-03-15 07:58:03,174 INFO IEV: Europe
2022-03-15 07:58:03,187 INFO ASEA: SE Asia


### Grandma Valuation

In [3]:
df_valuation_metrics, d_fig = batchValuation(d_instrument_data, valuate_parameters={'value_by_years':True}, verbose=0, printfunc=logPrint)
df_valuation_metrics['name'] = df_valuation_metrics['ticker'].apply(lambda s: d_instrument[s])

df_valuation_metrics

,ticker,r2_train,train_years,annualized_return,current_price,fair_price,over_value_range,over_value_years,name
0,IVV,0.971477,9.99726,0.134574,429.309998,414.012927,0.036948,0.274557,SP500
1,3073.HK,0.915595,10.0,0.082654,45.200001,52.574816,-0.140273,-1.159404,Greater China
2,VPL,0.88908,9.99726,0.067192,71.470001,77.261859,-0.074964,-0.503701,Developed Asia-Pacific
3,IEV,0.779355,9.99726,0.052396,47.970001,49.258805,-0.026164,-0.137089,Europe
4,ASEA,0.352563,9.99726,0.016697,15.34,14.388844,0.066104,3.959114,SE Asia


### Grandma Portfolio Allocation

In [7]:
instruments_select = ['IVV', '3073.HK', 'VPL', 'ASEA', 'IEV']

df_portfolio = df_valuation_metrics[df_valuation_metrics['ticker'].isin(instruments_select)].copy()
df_portfolio = addCashPortfolio(df_portfolio, value_col='over_value_years')

d_instrument_prices = {ticker:data for ticker, data in d_instrument_data.items() if ticker in instruments_select}
d_weights = getCorrelationWeight(d_instrument_prices, with_cash=True, cash_name='cash', verbose=0)
df_portfolio['cor_weight'] = df_portfolio['ticker'].apply(lambda t: d_weights[t] )
weights = df_portfolio['cor_weight']
df_portfolio['portfolio'] = allocatePortfolio(df_portfolio['over_value_years'], scale=1, transformation='sigmoid', weights=weights)

df_portfolio


,ticker,r2_train,train_years,annualized_return,current_price,fair_price,over_value_range,over_value_years,name,cor_weight,portfolio
0,IVV,0.971477,9.99726,0.134574,429.309998,414.012927,0.036948,0.274557,SP500,0.140460,0.163104
1,3073.HK,0.915595,10.0,0.082654,45.200001,52.574816,-0.140273,-1.159404,Greater China,0.122397,0.250568
2,VPL,0.88908,9.99726,0.067192,71.470001,77.261859,-0.074964,-0.503701,Developed Asia-Pacific,0.101759,0.170581
3,IEV,0.779355,9.99726,0.052396,47.970001,49.258805,-0.026164,-0.137089,Europe,0.121211,0.174141
4,ASEA,0.352563,9.99726,0.016697,15.34,14.388844,0.066104,3.959114,SE Asia,0.347506,0.017497
5,cash,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.166667,0.224109
